In [1]:
1 + 2

3

In [1]:
pow(100,2)

10000

In [2]:
max(1, -2, 3, 5)

5

In [3]:
max(min(1, -2), min(pow(3,5), -4))

-2

In [5]:
from math import sqrt
sqrt(256)

16.0

In [6]:
from operator import add, sub, mul
add(14, 28)
sub(100, mul(7, add(8, 4)))

16

In [7]:
radius = 10
radius

10

In [8]:
def square(x):
    return mul(x, x)

In [9]:
square(10)

100

In [10]:
square

<function __main__.square>

- Each function should have exactly one job. That job should be identifiable with a short name and characterizable in a single line of text. Functions that perform multiple jobs in sequence should be divided into multiple functions
- Don't repeat yourself is a central tenet of software engineering. The so-called DIY principle states that multiple fragments of code should not describe redundant logic
- Functions should be defined generally.

### Iteration
Iteration control structures are another mechanism for executing the same statements many times.
Consider the sequence of Fibonacci numbers, in which each number is the sum of the preceding two:
0, 1, 1, 2, 3, 5, 8, 13, 21,....

In [11]:
def fib(n):
    """Compute the nth Fibanacci number, for n >= 2."""
    pred, curr = 0, 1
    k = 2
    while k < n:
        pred, curr = curr, pred + curr
        k = k + 1
    return curr

In [12]:
result = fib(8)

In [13]:
result

13

In [14]:
help(fib)

Help on function fib in module __main__:

fib(n)
    Compute the nth Fibanacci number, for n >= 2.



### Testing

In [15]:
assert

SyntaxError: invalid syntax (<ipython-input-15-54b8a87cebf1>, line 1)

In [16]:
assert fib(8) == 13, 'The 8th Fibonacci number should be 13'

In [19]:
def fib_test():
    assert fib(1) == 1, 'The 2nd Fibonacci number should 1'
    assert fib(3) == 1, 'The 3rd Fibonacci number should 1'
    assert fib(50) == 7778742049, 'Error at the 50th Fibonacci number'


In [20]:
fib_test()

In [21]:
from doctest import testmod

In [22]:
testmod()

TestResults(failed=0, attempted=0)

### HOF

#### Functions as Arguments

These three functions clearly share a common underlying pattern. They are for the most identical, differing only in name and the function of k used to compute the term to be added. We could generate each of the functions by filling in slots in the same template:


In [23]:
def sum_naturals(n):
    total, k = 0, 1
    while k <= n:
        total, k = total + k, k + 1
    return total

In [24]:
sum_naturals(100)

5050

In [27]:
def sum_cubes(n):
    total, k = 0, 1
    while k <= n:
        total, k = total + k*k*k, k + 1
    return total

In [28]:
sum_cubes(100)

25502500

In [1]:
def pi_sum(n):
    total, k = 0, 1
    while k <= n:
        total, k = total + 8 / ((4*k -3) * (4*k-1)), k + 1
    return total

In [2]:
pi_sum(100)

3.1365926848388144

#### Fill slot in same template:

In [2]:
def template(n, fn):
    total, k = 0, 1
    while k <= n:
        total, k = total + fn(k), k + 1
    return total

### Functions as General Methods

More powerful abstraction: some funtions express general methods of computation, independent of the particular functions they call. Consider the following example, which implements a general method for iterative improvement and uses it to compute the golden ratio.

A iterative improvement algorithm begins with a **guess** of a solution to an equation. It repeatedly applies an **update** fuction to improve that guess, and applies a **close** comparision to check whether the current **guess** is "close enough" to be considered corrent.

In [1]:
def improve(update, close, guess=1):
    while not close(guess):
        guess = update(guess)
    return guess

In [4]:
def golden_update(guess):
    return 1/guess + 1

def approx_eq(x, y, tolerance=1e-30):
    return abs(x - y) < tolerance

def square_close_to_successor(guess):
    return approx_eq(guess * guess, guess + 1)

In [5]:
improve(golden_update, square_close_to_successor)

1.618033988749895

The example illustrate two related big ideas in computer science. First, naming and functions allow us to abstract away a vast amount of complexity. While each function definition has been trivial, the computational process set in motion by our evalution procedure is quite intricate. Second, it is only by virtue of the fact that we have an extremely general evaluation for Python language that small components can be composed into complex processes.

### Defining Functions III: Nested Definitions

The above examples demostrate how the ability to pass functions as arguments significantly enhances the expressive power of our programming language. Each general concept or equation maps onto its own short function. One negative consequence of this approach is that the global frame becomes cluttered with names of small functions, which must all be unique. Another problem is that we are constrained by particular function signatures: the **update** argument to **improve** must take exactly one argument. Nested function defintions address both of these problems, but require use to enrich our environment model.

Computing the square root of a number:

In [6]:
def average(x, y):
    return (x + y) / 2
def sqrt_update(x, a):
    return average(x, a/x)

Notice above, this two-argument update function is incompatible
with **improve**, and it provides only a single update, while we really care about taking square roots by repeated updates. The solution to both of these issues is to place function definitions inside the body of other definitions.

In [7]:
def sqrt(a):
    def sqrt_update(x):
        return average(x, a/x)
    def sqrt_close(x):
        return approx_eq(x * x, a)
    return improve(sqrt_update, sqrt_close)

Like local assignment, local **def** statements only affect the current local frame. These functions are only in scope while **sqrt** is being evaluated. Consistent with our evaluation procedure, these local **def** statements don't even get evaluated until **sqrt** is called.

**Lexical scope**. Locally defined functions are also have access to the name bindings in the scope in which they are defined. In this example, **sqrt_update** refers to the name **a**, which is a formal parameter of its enclosing function **sqrt**. This discipline of sharing names among nested definitions is called *lexical scoping*. Critically, the inner functions have access to the names in the environment where they are defined(not where they are called).

We require two extensions to our environment model to enable lexical scoping.
1. Each user-defined function has a parent environment: the environment in which it was defined.
2. When a user-defined function is called, its local frame extends its parent environment.

In [8]:
sqrt(25)

5.0